In [18]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *


import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
experiment = Experiment.objects.last()

In [20]:
master_df = get_all_frames(experiment)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:01<00:00, 14.77it/s]


In [21]:
!ls /shared/outputs

infocompare  minitest  test


In [22]:
plot_all_models(master_df, notebook=True)

Loading BokehJS ...

In [ ]:
# s = ['_mlp', '_mlp_1', '_mlp_2', '_lstm', '_lstm_1', '_lstm_2', 'Straightforward']
fig = compare_best_responses(master_df)
special_save_fig(fig, "configs.png", fmt=None, dpi=300, tight=True)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))
plt.imshow(list(q_network.output_layer.parameters())[0].data)
plt.colorbar()

In [ ]:
with open('/shared/outputs/feb7_large/mlp/evaluations/rewards_checkpoint_1500000_checkpoint_1500000_br_0_lstm_test.pkl', 'rb') as f:
    rewards = pickle.load(f)


In [ ]:
np.array(rewards['rewards'][0]).mean()

In [ ]:
master_df[['config', 'PositiveRegret']]

In [ ]:
x = master_df['config'].unique().tolist()
x.remove(None)
x.remove("Straightforward")
x

In [ ]:
for config in x:
    portfolio = [config, 'Straightforward']
    p = master_df.query('config in @portfolio and player == best_responder').copy()
    # NOT QUITE RIGHT
    p['frac'] = p['PositiveRegret'] / p['MaxPositiveRegret']
    p = p.groupby(['model', 't', 'player']).apply(lambda x: x['frac'].max())
    print(config, p.mean())

In [ ]:
y = list(x)
y.remove('_mlp_wide')
for config in y:
    portfolio = [config, '_mlp_wide', 'Straightforward']
    p = master_df.query('config in @portfolio and player == best_responder').copy()
    # NOT QUITE RIGHT
    p['frac'] = p['PositiveRegret'] / p['MaxPositiveRegret']
    p = p.groupby(['model', 't', 'player']).apply(lambda x: x['frac'].max())
    print(config, p.mean())

In [ ]:
z = list(y)
z.remove('_mlp_deep_wide')
for config in z:
    portfolio = [config, '_mlp_deep_wide', '_mlp_wide', 'Straightforward']
    p = master_df.query('config in @portfolio and player == best_responder').copy()
    # NOT QUITE RIGHT
    p['frac'] = p['PositiveRegret'] / p['MaxPositiveRegret']
    p = p.groupby(['model', 't', 'player']).apply(lambda x: x['frac'].max())
    print(config, p.mean())

In [4]:
dir(bokeh.plotting)

['Column',
 'ColumnDataSource',
 'DEFAULT_TOOLS',
 'Document',
 'Figure',
 'GMap',
 'GridSpec',
 'Row',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_decorators',
 '_docstring',
 '_figure',
 '_graph',
 '_legends',
 '_plot',
 '_renderer',
 '_stack',
 '_tools',
 'annotations',
 'column',
 'curdoc',
 'figure',
 'from_networkx',
 'glyph_api',
 'gmap',
 'graph',
 'gridplot',
 'log',
 'logging',
 'markers',
 'output_file',
 'output_notebook',
 'reset_output',
 'row',
 'save',
 'show']

In [14]:
EquilibriumSolverRun.objects.last().pk

32